#Predict what makes for a successful bike station

In [45]:
from pandas import DataFrame, Series
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model

In [46]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (14.0, 6.0)

In [47]:
weather = pd.read_table('daily_weather.tsv')
usage = pd.read_table('usage_2012.tsv')
stations = pd.read_table('stations.tsv')

#Compute the number of rentals per station per day

In [48]:
days = usage[['station_start', 'time_start']]
days['day'] = days['time_start'].str[5:10]
days['count'] = 1

/Users/Scott/.virtualenvs/bike_share/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app
/Users/Scott/.virtualenvs/bike_share/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [52]:
new_stations = pd.merge(days, stations, left_on='station_start', right_on='station')

In [61]:
rentals_per_station = new_stations.groupby('station').size()

In [63]:
avg_rentals = rentals_per_station.apply(lambda x: x/366)
avg_rentals

station
10th & E St NW                                       17
10th & Monroe St NE                                  10
10th & U St NW                                       71
10th St & Constitution Ave NW                        55
11th & H St NE                                       35
11th & Kenyon St NW                                  61
12th & Army Navy Dr                                  10
12th & L St NW                                       23
12th & Newton St NE                                   3
13th & D St NE                                       52
13th & H St NE                                       42
13th St & New York Ave NW                            53
14th & D St NW / Ronald Reagan Building              51
14th & D St SE                                       33
14th & G St NW                                       53
14th & Harvard St NW                                 71
14th & R St NW                                       68
14th & Rhode Island Ave NW              

###Our stations data has a huge number of quantitative attributes: fast_food, parking, restaurant, etc... Some of them are encoded as 0 or 1 (for absence or presence), others represent counts. To start with, run a simple linear regression where the input (x) variables are all the various station attributes and the output (y) variable is the average number of rentals per day.

In [ ]:
regr = linear_model.LinearRegression()
regr.fit()